In [2]:
import _pickle as pkl
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import shapely.geometry as geo
import numpy as np
import networkx as nx
import pandas as pd
from scipy.optimize import minimize

In [3]:
# # OSM
# nodes = pd.read_csv('devens_osm/osm_nodes.txt', header=None)
# edges = np.genfromtxt('devens_osm/osm_edges.txt', delimiter=',', dtype=int)
# # Devens map (GT)
# devens = pkl.load(open('devens_map/devens_map_poly.pkl', 'rb'), encoding = 'bytes')
# # # LiDAR scans
# # df = pkl.load(open('pointclouds/scans_in_utm_small.pkl', 'r'))
# df = pd.read_pickle('pointclouds/scans_in_utm_small.pkl')

In [22]:
# File paths
devens_map_file = '/home/sai/TopometricRegistration/devensData/devens_map/devens_map_poly.pkl'
osm_nodes_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_nodes.txt'
osm_edges_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_edges.txt'
cloud_file = '/home/sai/Downloads/scans_in_utm_annotated_small.pkl'
plot_dir = '/home/sai/TopometricRegistration/cache/'

ceres_build_dir = '/home/sai/TopometricRegistration/ceresCode/build/'
ceres_input_dir = '/home/sai/TopometricRegistration/ceresCode/data/'

In [23]:
# Load in data
devens_map = pkl.load(open(devens_map_file, 'rb'), encoding = 'bytes')
osm_nodes = np.genfromtxt(osm_nodes_file, delimiter = ',')
osm_edges = np.genfromtxt(osm_edges_file, delimiter = ',', dtype = int)
cloud = pd.read_pickle(cloud_file)

In [5]:
# # Construct a graph (OSM)
# G = nx.Graph()
# #G.add_nodes_from(range(len(nodes)))
# i = 0
# for row in nodes.iterrows():
#     G.add_node(i, x=row[1][0], y=row[1][1])
#     i = i+1

In [24]:
scan_idx = 0

scan = cloud.iloc[scan_idx]['scan_utm']
road_mask = cloud.iloc[scan_idx]['is_road_truth']
road_mask = np.where(road_mask)
xmin = min(scan['x'])
xmax = max(scan['x'])
ymin = min(scan['y'])
ymax = max(scan['y'])
# print(cloud.iloc[0]['is_road_truth'])
# print(xmin, xmax, ymin, ymax)
# plt.plot(scan['x'], scan['y'], '.', ms=1)

# plot_coords = lambda obj: plt.plot(obj.xy[0],obj.xy[1], 'k')
# plot_coords(devens_map.exterior)
# [plot_coords(x) for x in devens_map.interiors]
# plt.plot((xmin+xmax)/2,(ymin+ymax)/2,'gx')

# Extract OSM nodes that are within the distance threshold
osm_thresh = 20.
xinds = np.where(np.logical_and(osm_nodes[:,0] > xmin, osm_nodes[:,0] < xmax))
yinds = np.where(np.logical_and(osm_nodes[:,1] > ymin, osm_nodes[:,1] < ymax))
osm_inds = np.intersect1d(xinds, yinds)

# Active set of OSM nodes
osm_nodes_active = osm_nodes[osm_inds]

# Extract LiDAR scan points that correspond to road
scan_active = scan[road_mask]
# print(scan_active, scan_active.shape)
scan_active = np.asarray([[item[0] for item in scan_active], [item[1] for item in scan_active]]).T

In [30]:
print(scan_active.shape)
print(osm_nodes_active.shape)

(22784, 2)
(17, 2)


In [7]:
xs = nx.get_node_attributes(G, 'x')
ys = nx.get_node_attributes(G, 'y')
#print(xs[0])
G.add_edges_from(edges)
scan = df.iloc[0]['scan_utm']
pcl_scan = np.zeros((2, 3, 111360))
#print(scan['x'].shape)
pcl_coor = np.zeros((2,2))
for i in range(2):
    pcl_scan[i,0,:] = scan['x']
    pcl_scan[i,1,:] = scan['y']
    pcl_scan[i,2,:] = scan['z']
    pcl_coor[i,:] = df.iloc[i]['x'], df.iloc[i]['y']

In [9]:
count1 = 0
count2 = 0
road_pts = np.zeros((1000, 2))
for i in range(111360):
    if pcl_scan[0,1,i] < -2.5:
        if count1 < 1000:
            road_pts[count1, 0] = pcl_scan[0,0,i]
            road_pts[count1, 1] = pcl_scan[0,2,i]
        count1 = count1 + 1

    if pcl_scan[0,1,i] < 0:
        count2 = count2 + 1
print(count1, count2)

33628 44182


In [10]:
def transform_devens_2d(x):
    devens_new = np.zeros((devens_arr.shape[0], 2))
    for i in range(devens_arr.shape[0]):
        x_new = np.cos(x[0]) * devens_arr[i,0] + np.sin(x[0]) * devens_arr[i,1] + x[1]
        y_new = -np.sin(x[0]) * devens_arr[i,0] + np.cos(x[0]) * devens_arr[i,1] + x[2]

        devens_new[i,0] = x_new
        devens_new[i,1] = y_new
    return devens_new

In [11]:
def find_closest(x_new, y_new):
    closest = 100000.0
    for j in range(len(xs)):
        dist = np.sqrt((x_new - xs[j])**2 + (y_new - ys[j])**2)
        if dist == 0:
            return dist
        if dist < closest:
            closest = dist

    return closest

In [13]:

def cost_func_2d(x):
    my_cost = 0
    for i in range(devens_arr.shape[0]//100):
        x_new = np.cos(x[0]) * devens_arr[i*100,0] + np.sin(x[0]) * devens_arr[i*100,1] + x[1]
        y_new = -np.sin(x[0]) * devens_arr[i*100,0] + np.cos(x[0]) * devens_arr[i*100,1] + x[2]
        my_cost = my_cost + find_closest(x_new, y_new)
    return my_cost

In [14]:

devens_exterior = np.asarray(devens.exterior.coords[:])
devens_interior = []
for interior in devens.interiors:
    devens_interior += interior.coords[:]
devens_interior = np.asarray(devens_interior)
devens_arr = np.concatenate((devens_exterior, devens_interior), axis = 0)

In [15]:
devens_arr = np.zeros((1000,2))
devens_arr = road_pts
#print(devens.exterior.coords[1,0])
#x0 = np.array([0.5, 0.1, 0.1, 0.5, 1.0, 0.8, 1.2])
x0 = np.array([0.1, 0.8, 1.2])
res = minimize(cost_func_2d, x0, method='nelder-mead', options={'xtol':1e-8, 'disp':True})
#res_bounded = minimize(cost_func_2d, x0, method='trust-constr', options={'verbose':1}, bounds=bounds)
#print(res.x)
devens_trans = transform_devens_2d(res.x)

Optimization terminated successfully.
         Current function value: 9.185748
         Iterations: 131
         Function evaluations: 263


In [19]:
plt.scatter(devens_trans[:,0], devens_trans[:,1])
plt.show()
plt.savefig('/home/sai/TopometricRegistration/devensData/sai_devens.png')
plt.scatter(road_pts[:,0], road_pts[:,1])
plt.show()
plt.savefig('/home/sai/TopometricRegistration/devensData/sai_road.png')